In [1]:
import torch,os,time
import torch.optim as optim
import torch.nn as nn
from torch.autograd import Variable
from torchvision import transforms,datasets
import numpy as np
from utils.data import RandomNoiseGenerator,Data
from utils.train_history import train_history
from utils import visualizer,util
import itertools
from models.model import Generator, Discriminator
from IPython.core.debugger import set_trace
# from models.model import Generator_test, Discriminator_test
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
device = torch.device('cuda:2' if torch.cuda.is_available() else 'cpu')
if torch.backends.cudnn.enabled:
    torch.backends.cudnn.benchmark = True
device

device(type='cuda', index=2)

In [3]:
latent_size = 512
target_resol = 256
first_resol = 16
use_sigmoid = False
train_kimg = 600
train_img = train_kimg*1000
transition_kimg = 600
transition_img = transition_kimg*1000

g_lr_max =  0.0002
d_lr_max =  0.0002

beta1 = 0.5
beta2 = 0.999

lambda_A=10
lambda_B=10
lambda_recon=2
lambda_idt=0.5

report_it = 400
show_it = 400
save_it=400

In [4]:
display_id=1
display_winsize=256
display_ncols=3
display_server='http://localhost'
display_port=8097
display_env='pggan3'
results_dir='results'
project_name = 'pggan3'
project_dir=os.path.join(results_dir,project_name)

vis = visualizer.Visualizer(display_id,display_winsize,display_ncols,display_server,display_port,display_env,
                 project_name,results_dir)

if not os.path.isdir(project_dir):
    os.makedirs(project_dir)

create web directory results/pggan3/web...


In [5]:
real_dir = '/data/persona_cyclegan/faces_wo_bg/train'
# anime_dir = '/data/persona_cyclegan/anime_mix_face_3'
anime_dir = '/data/persona_cyclegan/anime_wo_bg/train'
# real_dir = '/data/persona_cyclegan/real_mix_face_2'

# real_dir = '/data/persona_cyclegan/real_test'
# anime_dir = '/data/persona_cyclegan/anime_test'
data_A = Data(real_dir)
data_B = Data(anime_dir)
print(data_A.get_count())
print(data_B.get_count())

35997
27280


In [6]:
# real_dir = '/data/persona_cyclegan/faces_wo_bg'
# anime_dir = '/data/persona_cyclegan/anime'
# def load_imgs(src_data_path, tgt_data_path, fine_size):
# #     load_size = fine_size * 9//8
#     transform = transforms.Compose([
#             transforms.Resize((fine_size, fine_size)),
# #             transforms.RandomCrop(fine_size),
#             transforms.RandomHorizontalFlip(),
#             transforms.ColorJitter(0.1,0.1,0.1,0.1),
#             transforms.ToTensor(),
#             transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
#     ])
    
#     train_loader_A = torch.utils.data.DataLoader(datasets.ImageFolder(src_data_path, transform), batch_size=32, shuffle=True, drop_last=True)
#     train_loader_B = torch.utils.data.DataLoader(datasets.ImageFolder(tgt_data_path, transform), batch_size=32, shuffle=True, drop_last=True)
#     return train_loader_A,train_loader_B

# train_loader_A,train_loader_B = load_imgs(real_dir, anime_dir, 8)

In [7]:
G_A = Generator(num_channels=3, latent_size=latent_size, resolution=target_resol, fmap_max=latent_size, fmap_base=8192, tanh_at_end=True,resnets=1)
G_B = Generator(num_channels=3, latent_size=latent_size, resolution=target_resol, fmap_max=latent_size, fmap_base=8192, tanh_at_end=True,resnets=1)
# G_A = Generator_test()
# G_B = Generator_test()

D_A = Discriminator(num_channels=3, mbstat_avg='all', resolution=target_resol, fmap_max=latent_size, fmap_base=8192, sigmoid_at_end=False)
D_B = Discriminator(num_channels=3, mbstat_avg='all', resolution=target_resol, fmap_max=latent_size, fmap_base=8192, sigmoid_at_end=False)
# D_A = Discriminator_test()
# D_B = Discriminator_test()

print(G_A)
print(D_A)
G_A,G_B,D_A,D_B = G_A.to(device),G_B.to(device),D_A.to(device),D_B.to(device)
optim_G = optim.Adam(itertools.chain(G_A.parameters(),G_B.parameters()), g_lr_max, betas=(beta1, beta2))
optim_D_A = optim.Adam(D_A.parameters(), d_lr_max, betas=(beta1, beta2))
optim_D_B = optim.Adam(D_B.parameters(), d_lr_max, betas=(beta1, beta2))

all_models = {'G_A.pkl':G_A,
             'G_B.pkl':G_B,
             'D_A.pkl':D_A,
             'D_B.pkl':D_B}
            

print('---------- Networks initialized -------------')
for model_name,model in [('G',G_A),('D',D_A)]:
    num_params = 0
    for param in model.parameters():
        num_params += param.numel()
    print(str.format('{} has {} number of parameters', model_name, num_params))
print('-----------------------------------------------')

Generator(
  (output_layer): GSelectLayer(
    (pre): ModuleList(
      (0): Sequential(
        (0): ReflectionPad2d((3, 3, 3, 3))
        (1): Conv2d(3, 512, kernel_size=(7, 7), stride=(1, 1))
        (2): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
        (3): ReLU(inplace)
      )
      (1): Sequential(
        (0): ReflectionPad2d((3, 3, 3, 3))
        (1): Conv2d(3, 512, kernel_size=(7, 7), stride=(1, 1))
        (2): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
        (3): ReLU(inplace)
      )
      (2): Sequential(
        (0): ReflectionPad2d((3, 3, 3, 3))
        (1): Conv2d(3, 512, kernel_size=(7, 7), stride=(1, 1))
        (2): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
        (3): ReLU(inplace)
      )
      (3): Sequential(
        (0): ReflectionPad2d((3, 3, 3, 3))
        (1): Conv2d(3, 512, kernel_size=(7, 7), stride=(1, 1))
        (2): BatchNorm2d(

---------- Networks initialized -------------
G has 65890901 number of parameters
D has 30900353 number of parameters
-----------------------------------------------


In [8]:
rampup_kimg = 10000
rampdown_kimg = 10000
total_kimg = 10000

def _rampup(epoch, rampup_length):
    if epoch < rampup_length:
        p = max(0.0, float(epoch)) / float(rampup_length)
        p = 1.0 - p
        return np.exp(-p*p*5.0)
    else:
        return 1.0

def _rampdown_linear(epoch, num_epochs, rampdown_length):
    if epoch >= num_epochs - rampdown_length:
        return float(num_epochs - epoch) / rampdown_length
    else:
        return 1.0

In [9]:
def get_bs(resolution):
    R = int(np.log2(resolution))
    if R < 7:
        bs = 32 / 2**(max(0, R-2))
    else:
        bs = 8 / 2**(min(2, R-7))
    return int(bs)

bs_map = {2**R: get_bs(2**R) for R in range(2, 11)}

In [10]:
train_hist = train_history(['G_gan_loss',
                              'G_idt_loss',
                              'G_cycle_loss',
                              'D_A_real_loss',
                              'D_A_fake_loss',
                              'D_B_real_loss',                                         
                              'D_B_fake_loss',                                         
                              ])

In [11]:
L1_loss = nn.L1Loss().to(device) 
GAN_loss = nn.MSELoss().to(device)
# GAN_loss = nn.BCELoss().to(device)

In [12]:
def save_models(models, folder):
    for k, v in models.items():
        torch.save(v.state_dict(), os.path.join(folder, k))

In [13]:
all_models = {'G_A.pkl':G_A,
             'G_B.pkl':G_B,
             'D_A.pkl':D_A,
             'D_B.pkl':D_B}

def load_models(models, folder):
    for k, v in models.items():
        v.load_state_dict(torch.load(os.path.join(folder, k)))
        
load_models(all_models, os.path.join(results_dir,project_name,'2'))

## Train

In [14]:
to_level = int(np.log2(target_resol))
from_level = int(np.log2(first_resol))

for model in [G_A,G_B,D_A,D_B]:
    model.train()

for R in range(from_level-1, to_level):
    batch_size = bs_map[2 ** (R+1)]
    phases = {'stabilize':[0, train_img//batch_size], 'fade_in':[train_img//batch_size+1, (transition_img+train_img)//batch_size]}
    if R==to_level-1:
        del phases['fade_in']
    print('starting scale %d, batch_size is %d, and the iterations in each phase is %d'%(R,batch_size,train_img//batch_size))
#     for phase in ['fade_in']: 
    for phase in ['stabilize', 'fade_in']:
        num_pool = 80
        fake_A_pool = util.ImagePool(num_pool)
        fake_B_pool = util.ImagePool(num_pool)
        if phase in phases:
            _range = phases[phase]
            from_it = _range[0]
            cur_it = 0
            total_it = _range[1]
            remaining_it = total_it-from_it
            cur_nimg = _range[0]*batch_size
            resol = 2 ** (R+1)
            
            previous_time = time.time()
            phase_start_time = time.time()
            
            for it in range(from_it, total_it):
#             print(len(train_loader_A))
#             print(len(train_loader_B))
#             for it,((real_A,_),(real_B,_)) in enumerate(zip(train_loader_A, train_loader_B)):
#                 set_trace()
                cur_it += 1
                if phase == 'stabilize':
                    cur_level = R
                else:
                    cur_level = R + cur_it/remaining_it
                cur_resol = 2 ** int(np.ceil(cur_level+1))
                cur_nimg += batch_size
                cur_scale = R+cur_it/total_it/2
                if phase == 'fade_in':
                    cur_scale+=0.5

                real_A = data_A.next(batch_size,cur_resol*2,cur_level+1)             
                real_B = data_B.next(batch_size,cur_resol*2,cur_level+1)
                real_A,real_B = real_A.to(device), real_B.to(device)
                # ===preprocess===
#                 for param_group in optim_G.param_groups:
#                     lrate_coef = _rampup(cur_nimg / 1000.0, rampup_kimg)
#                     lrate_coef *= _rampdown_linear(cur_nimg / 1000.0,total_kimg, rampdown_kimg)
#                     param_group['lr'] = lrate_coef * g_lr_max
  
#                 for param_group in optim_D_A.param_groups:
#                     lrate_coef = _rampup(cur_nimg / 1000.0, rampup_kimg)
#                     lrate_coef *= _rampdown_linear(cur_nimg / 1000.0, total_kimg, rampdown_kimg)
#                     param_group['lr'] = lrate_coef * d_lr_max
#                 for param_group in optim_D_B.param_groups:
#                     lrate_coef = _rampup(cur_nimg / 1000.0, rampup_kimg)
#                     lrate_coef *= _rampdown_linear(cur_nimg / 1000.0, total_kimg, rampdown_kimg)
#                     param_group['lr'] = lrate_coef * d_lr_max
                    
                
                # ===update G===
                for model in [D_A,D_B]:
                    for param in model.parameters():
                        param.requires_grad = False
                
                #G_A
                fake_B, G_model_status = G_A(real_A, cur_level)
                d_fake_A, D_model_status = D_B(fake_B, cur_level)
                d_fake_A_loss = GAN_loss(d_fake_A, torch.ones(d_fake_A.size(), device=device))
                
                sim_A_loss = L1_loss(fake_B,real_A) * lambda_idt
                
                recon_A,_ = G_B(fake_B,cur_level)
                recon_A_loss = L1_loss(recon_A,real_A) * lambda_recon
                
                G_A_loss = (d_fake_A_loss+sim_A_loss+recon_A_loss)*lambda_A
                
                
                fake_A,_ = G_B(real_B,cur_level)
                d_fake_B,_ = D_A(fake_A,cur_level)
                d_fake_B_loss = GAN_loss(d_fake_B, torch.ones(d_fake_B.size(), device=device))

                sim_B_loss = L1_loss(fake_A,real_B)*lambda_idt
                
                recon_B,_ = G_A(fake_A,cur_level)
                recon_B_loss = L1_loss(recon_B,real_B)*lambda_recon
                
                G_B_loss = (d_fake_B_loss + sim_B_loss + recon_B_loss)*lambda_B
                
                G_loss = G_A_loss+G_B_loss
                
                optim_G.zero_grad()
                G_loss.backward()
                optim_G.step()
                
                G_gan_loss = d_fake_A_loss + d_fake_B_loss
                G_idt_loss = sim_A_loss + sim_B_loss
                G_recon_loss = recon_A_loss + recon_B_loss

                # ===generate sample images===
                if it % show_it == 0:
                    save_result = it % save_it == 0
                    vis.display_current_results([real_A,fake_B,recon_A,real_B,fake_A,recon_B],cur_scale,save_result)
                    
#                 ===update D===
                
                for model in [D_A,D_B]:
                    for param in model.parameters():
                        param.requires_grad = True
                #D_B
                fake_B,_ = G_A(real_A, cur_level=cur_level)
                fake_B_from_pool = fake_B_pool.query(fake_B.detach())
                d_real_B,_ = D_B(real_B, cur_level=cur_level)
                d_fake_B,_ = D_B(fake_B_from_pool, cur_level=cur_level)

                d_real_B_loss = GAN_loss(d_real_B, torch.ones(d_real_B.size(), device=device))
                d_fake_B_loss = GAN_loss(d_fake_B, torch.zeros(d_fake_B.size(), device=device))
#                 d_real_B_loss = GAN_loss(d_real_B, 1-torch.rand(d_real_B.size(), device=device)/10.0)
#                 d_fake_B_loss = GAN_loss(d_fake_B, torch.rand(d_real_B.size(), device=device)/10.0)        

                d_loss_B = 0.5 * (d_real_B_loss + d_fake_B_loss)
                
                optim_D_B.zero_grad()
                d_loss_B.backward()
                optim_D_B.step()

                #D_A
                fake_A,_ = G_B(real_B, cur_level=cur_level)
                fake_A_from_pool = fake_A_pool.query(fake_A.detach())
                d_real_A,_ = D_A(real_A, cur_level=cur_level)
                d_fake_A,_ = D_A(fake_A_from_pool, cur_level=cur_level)

                d_real_A_loss = GAN_loss(d_real_A, torch.ones(d_real_A.size(), device=device))
                d_fake_A_loss = GAN_loss(d_fake_A, torch.zeros(d_fake_A.size(), device=device))             
#                 d_real_A_loss = GAN_loss(d_real_A, 1-torch.rand(d_real_B.size(), device=device)/10.0)
#                 d_fake_A_loss = GAN_loss(d_fake_A, torch.rand(d_real_B.size(), device=device)/10.0)      

                d_loss_A = 0.5 * (d_real_A_loss + d_fake_A_loss)
    
                optim_D_A.zero_grad()
                d_loss_A.backward()    
                optim_D_A.step()
            
            

                # ===report ===
                train_hist.add_params([G_gan_loss,G_idt_loss,G_recon_loss,d_real_A_loss,d_fake_A_loss,d_real_B_loss,d_fake_B_loss])
                if it% report_it == 0:                    
                    phase_time = time.time() - phase_start_time
                    losses = train_hist.check_current_avg()
                    it_time = time.time() - previous_time
                    vis.print_current_losses(cur_level, it, losses, it_time, phase_time)
                    vis.plot_current_losses(cur_scale, losses)
                    previous_time = time.time()
                    print(G_model_status)
                    print(D_model_status)
                    
                
                # ===save model===
                if it % save_it == 0:
                    save_models(all_models,project_dir)

    model_folder_at_scale = os.path.join(project_dir,str(R))
    if not os.path.isdir(model_folder_at_scale):
        os.makedirs(model_folder_at_scale)
    save_models(all_models,model_folder_at_scale)

starting scale 3, batch_size is 8, and the iterations in each phase is 75000
(current_level: 3.000, iters: 0, time_from_last_report: 2.783, since_the_phase_start: 2.753) G_gan_loss: 0.477 G_idt_loss: 0.319 G_cycle_loss: 0.529 D_A_real_loss: 0.206 D_A_fake_loss: 0.331 D_B_real_loss: 0.214 D_B_fake_loss: 0.345 
{'pre_used': [2], 'enchain_used': [2, 1, 0], 'dechain_used': [0, 1, 2], 'post_used': [2]}
{'inputs_used': [4], 'chain_used': [4, 5, 6]}
(current_level: 3.000, iters: 400, time_from_last_report: 201.006, since_the_phase_start: 203.529) G_gan_loss: 0.559 G_idt_loss: 0.308 G_cycle_loss: 0.550 D_A_real_loss: 0.252 D_A_fake_loss: 0.251 D_B_real_loss: 0.252 D_B_fake_loss: 0.251 
{'pre_used': [2], 'enchain_used': [2, 1, 0], 'dechain_used': [0, 1, 2], 'post_used': [2]}
{'inputs_used': [4], 'chain_used': [4, 5, 6]}
(current_level: 3.000, iters: 800, time_from_last_report: 200.631, since_the_phase_start: 404.165) G_gan_loss: 0.559 G_idt_loss: 0.305 G_cycle_loss: 0.547 D_A_real_loss: 0.250 D

(current_level: 3.000, iters: 8800, time_from_last_report: 201.707, since_the_phase_start: 4416.736) G_gan_loss: 0.543 G_idt_loss: 0.306 G_cycle_loss: 0.489 D_A_real_loss: 0.255 D_A_fake_loss: 0.254 D_B_real_loss: 0.253 D_B_fake_loss: 0.253 
{'pre_used': [2], 'enchain_used': [2, 1, 0], 'dechain_used': [0, 1, 2], 'post_used': [2]}
{'inputs_used': [4], 'chain_used': [4, 5, 6]}
(current_level: 3.000, iters: 9200, time_from_last_report: 201.921, since_the_phase_start: 4618.665) G_gan_loss: 0.538 G_idt_loss: 0.306 G_cycle_loss: 0.488 D_A_real_loss: 0.254 D_A_fake_loss: 0.254 D_B_real_loss: 0.253 D_B_fake_loss: 0.253 
{'pre_used': [2], 'enchain_used': [2, 1, 0], 'dechain_used': [0, 1, 2], 'post_used': [2]}
{'inputs_used': [4], 'chain_used': [4, 5, 6]}
(current_level: 3.000, iters: 9600, time_from_last_report: 201.097, since_the_phase_start: 4819.748) G_gan_loss: 0.542 G_idt_loss: 0.303 G_cycle_loss: 0.492 D_A_real_loss: 0.253 D_A_fake_loss: 0.252 D_B_real_loss: 0.254 D_B_fake_loss: 0.254 
{'

(current_level: 3.000, iters: 17600, time_from_last_report: 200.174, since_the_phase_start: 8831.171) G_gan_loss: 0.548 G_idt_loss: 0.304 G_cycle_loss: 0.463 D_A_real_loss: 0.254 D_A_fake_loss: 0.253 D_B_real_loss: 0.254 D_B_fake_loss: 0.254 
{'pre_used': [2], 'enchain_used': [2, 1, 0], 'dechain_used': [0, 1, 2], 'post_used': [2]}
{'inputs_used': [4], 'chain_used': [4, 5, 6]}
(current_level: 3.000, iters: 18000, time_from_last_report: 199.860, since_the_phase_start: 9031.035) G_gan_loss: 0.542 G_idt_loss: 0.304 G_cycle_loss: 0.461 D_A_real_loss: 0.256 D_A_fake_loss: 0.255 D_B_real_loss: 0.253 D_B_fake_loss: 0.253 
{'pre_used': [2], 'enchain_used': [2, 1, 0], 'dechain_used': [0, 1, 2], 'post_used': [2]}
{'inputs_used': [4], 'chain_used': [4, 5, 6]}
(current_level: 3.000, iters: 18400, time_from_last_report: 199.695, since_the_phase_start: 9230.737) G_gan_loss: 0.542 G_idt_loss: 0.302 G_cycle_loss: 0.459 D_A_real_loss: 0.254 D_A_fake_loss: 0.253 D_B_real_loss: 0.255 D_B_fake_loss: 0.254 

(current_level: 3.000, iters: 26400, time_from_last_report: 200.538, since_the_phase_start: 13253.619) G_gan_loss: 0.541 G_idt_loss: 0.299 G_cycle_loss: 0.445 D_A_real_loss: 0.254 D_A_fake_loss: 0.253 D_B_real_loss: 0.253 D_B_fake_loss: 0.253 
{'pre_used': [2], 'enchain_used': [2, 1, 0], 'dechain_used': [0, 1, 2], 'post_used': [2]}
{'inputs_used': [4], 'chain_used': [4, 5, 6]}
(current_level: 3.000, iters: 26800, time_from_last_report: 200.483, since_the_phase_start: 13454.110) G_gan_loss: 0.544 G_idt_loss: 0.298 G_cycle_loss: 0.441 D_A_real_loss: 0.256 D_A_fake_loss: 0.255 D_B_real_loss: 0.254 D_B_fake_loss: 0.254 
{'pre_used': [2], 'enchain_used': [2, 1, 0], 'dechain_used': [0, 1, 2], 'post_used': [2]}
{'inputs_used': [4], 'chain_used': [4, 5, 6]}
(current_level: 3.000, iters: 27200, time_from_last_report: 200.413, since_the_phase_start: 13654.534) G_gan_loss: 0.541 G_idt_loss: 0.301 G_cycle_loss: 0.439 D_A_real_loss: 0.255 D_A_fake_loss: 0.254 D_B_real_loss: 0.254 D_B_fake_loss: 0.2

(current_level: 3.000, iters: 35200, time_from_last_report: 202.193, since_the_phase_start: 17694.199) G_gan_loss: 0.542 G_idt_loss: 0.301 G_cycle_loss: 0.428 D_A_real_loss: 0.255 D_A_fake_loss: 0.255 D_B_real_loss: 0.254 D_B_fake_loss: 0.253 
{'pre_used': [2], 'enchain_used': [2, 1, 0], 'dechain_used': [0, 1, 2], 'post_used': [2]}
{'inputs_used': [4], 'chain_used': [4, 5, 6]}
(current_level: 3.000, iters: 35600, time_from_last_report: 206.378, since_the_phase_start: 17900.587) G_gan_loss: 0.545 G_idt_loss: 0.300 G_cycle_loss: 0.431 D_A_real_loss: 0.254 D_A_fake_loss: 0.253 D_B_real_loss: 0.254 D_B_fake_loss: 0.253 
{'pre_used': [2], 'enchain_used': [2, 1, 0], 'dechain_used': [0, 1, 2], 'post_used': [2]}
{'inputs_used': [4], 'chain_used': [4, 5, 6]}
(current_level: 3.000, iters: 36000, time_from_last_report: 201.083, since_the_phase_start: 18101.669) G_gan_loss: 0.545 G_idt_loss: 0.295 G_cycle_loss: 0.428 D_A_real_loss: 0.255 D_A_fake_loss: 0.254 D_B_real_loss: 0.253 D_B_fake_loss: 0.2

...

(current_level: 3.115, iters: 83600, time_from_last_report: 614.398, since_the_phase_start: 13195.124) G_gan_loss: 1.781 G_idt_loss: 0.305 G_cycle_loss: 0.533 D_A_real_loss: 0.020 D_A_fake_loss: 0.017 D_B_real_loss: 0.052 D_B_fake_loss: 0.045 
{'pre_used': [3, 3, 2], 'enchain_used': [3, 2, 1, 0], 'dechain_used': [0, 1, 2, 3], 'post_used': [2, 3]}
{'inputs_used': [3], 'chain_used': [3, 4, 5, 6]}
(current_level: 3.120, iters: 84000, time_from_last_report: 614.034, since_the_phase_start: 13809.228) G_gan_loss: 1.818 G_idt_loss: 0.318 G_cycle_loss: 0.534 D_A_real_loss: 0.018 D_A_fake_loss: 0.014 D_B_real_loss: 0.036 D_B_fake_loss: 0.029 
{'pre_used': [3, 3, 2], 'enchain_used': [3, 2, 1, 0], 'dechain_used': [0, 1, 2, 3], 'post_used': [2, 3]}
{'inputs_used': [3], 'chain_used': [3, 4, 5, 6]}
(current_level: 3.125, iters: 84400, time_from_last_report: 613.499, since_the_phase_start: 14422.745) G_gan_loss: 1.739 G_idt_loss: 0.311 G_cycle_loss: 0.538 D_A_real_loss: 0.023 D_A_fake_loss: 0.023 D_B

(current_level: 3.227, iters: 92000, time_from_last_report: 613.253, since_the_phase_start: 26085.395) G_gan_loss: 1.581 G_idt_loss: 0.307 G_cycle_loss: 0.509 D_A_real_loss: 0.025 D_A_fake_loss: 0.027 D_B_real_loss: 0.112 D_B_fake_loss: 0.094 
{'pre_used': [3, 3, 2], 'enchain_used': [3, 2, 1, 0], 'dechain_used': [0, 1, 2, 3], 'post_used': [2, 3]}
{'inputs_used': [3], 'chain_used': [3, 4, 5, 6]}
(current_level: 3.232, iters: 92400, time_from_last_report: 613.990, since_the_phase_start: 26699.428) G_gan_loss: 1.527 G_idt_loss: 0.294 G_cycle_loss: 0.508 D_A_real_loss: 0.041 D_A_fake_loss: 0.038 D_B_real_loss: 0.116 D_B_fake_loss: 0.090 
{'pre_used': [3, 3, 2], 'enchain_used': [3, 2, 1, 0], 'dechain_used': [0, 1, 2, 3], 'post_used': [2, 3]}
{'inputs_used': [3], 'chain_used': [3, 4, 5, 6]}
(current_level: 3.237, iters: 92800, time_from_last_report: 613.458, since_the_phase_start: 27312.932) G_gan_loss: 1.553 G_idt_loss: 0.303 G_cycle_loss: 0.531 D_A_real_loss: 0.025 D_A_fake_loss: 0.020 D_B

(current_level: 3.339, iters: 100400, time_from_last_report: 612.852, since_the_phase_start: 38972.080) G_gan_loss: 1.407 G_idt_loss: 0.278 G_cycle_loss: 0.508 D_A_real_loss: 0.044 D_A_fake_loss: 0.036 D_B_real_loss: 0.143 D_B_fake_loss: 0.131 
{'pre_used': [3, 3, 2], 'enchain_used': [3, 2, 1, 0], 'dechain_used': [0, 1, 2, 3], 'post_used': [2, 3]}
{'inputs_used': [3], 'chain_used': [3, 4, 5, 6]}
(current_level: 3.344, iters: 100800, time_from_last_report: 613.855, since_the_phase_start: 39585.929) G_gan_loss: 1.310 G_idt_loss: 0.298 G_cycle_loss: 0.519 D_A_real_loss: 0.064 D_A_fake_loss: 0.053 D_B_real_loss: 0.157 D_B_fake_loss: 0.148 
{'pre_used': [3, 3, 2], 'enchain_used': [3, 2, 1, 0], 'dechain_used': [0, 1, 2, 3], 'post_used': [2, 3]}
{'inputs_used': [3], 'chain_used': [3, 4, 5, 6]}
(current_level: 3.349, iters: 101200, time_from_last_report: 613.026, since_the_phase_start: 40199.014) G_gan_loss: 1.361 G_idt_loss: 0.289 G_cycle_loss: 0.520 D_A_real_loss: 0.050 D_A_fake_loss: 0.041 

(current_level: 3.451, iters: 108800, time_from_last_report: 613.430, since_the_phase_start: 51872.346) G_gan_loss: 1.107 G_idt_loss: 0.302 G_cycle_loss: 0.537 D_A_real_loss: 0.114 D_A_fake_loss: 0.094 D_B_real_loss: 0.174 D_B_fake_loss: 0.177 
{'pre_used': [3, 3, 2], 'enchain_used': [3, 2, 1, 0], 'dechain_used': [0, 1, 2, 3], 'post_used': [2, 3]}
{'inputs_used': [3], 'chain_used': [3, 4, 5, 6]}
(current_level: 3.456, iters: 109200, time_from_last_report: 613.501, since_the_phase_start: 52485.904) G_gan_loss: 1.100 G_idt_loss: 0.295 G_cycle_loss: 0.549 D_A_real_loss: 0.118 D_A_fake_loss: 0.101 D_B_real_loss: 0.163 D_B_fake_loss: 0.174 
{'pre_used': [3, 3, 2], 'enchain_used': [3, 2, 1, 0], 'dechain_used': [0, 1, 2, 3], 'post_used': [2, 3]}
{'inputs_used': [3], 'chain_used': [3, 4, 5, 6]}
(current_level: 3.461, iters: 109600, time_from_last_report: 613.029, since_the_phase_start: 53098.947) G_gan_loss: 1.087 G_idt_loss: 0.314 G_cycle_loss: 0.546 D_A_real_loss: 0.115 D_A_fake_loss: 0.102 

KeyboardInterrupt: 